# **Bias mitigation with debiasing exposure**

This demo  will show how to implement the post-processing "debiasing exposure" bias mitigator algorithm to enhance the fairness of a recommender system's output.

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

In [5]:
# Base Imports
import pandas as pd
import numpy as np
from holisticai.datasets.synthetic.recruitment import generate_rankings
from holisticai.bias.mitigation.postprocessing.debiasing_exposure.algorithm_utils import exposure_metric
from holisticai.bias.mitigation.postprocessing import DebiasingExposure

np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")

The dataset that we will use is a synthetic ranking dataset generated following the procedure described by Yang and Stoyanovich in their [research](https://arxiv.org/abs/1610.08559). This algorithm creates a ranked output of protected and unprotected candidates with a certain probability.

In [3]:
# Synthetic data
M = 1000
top_n = 20
p = 0.25
rankings = generate_rankings(M, top_n, p, return_p_attr=False)

baseline = exposure_metric(rankings, group_col='protected', query_col='X', score_col='score')
baseline

,Value
exposure_ratio,33221.217929
exposure difference,0.048336


## **Bias mitigation**

Apply the debiasing exposure mitigator algorithm.

In [6]:
# create the DebiasingExposure class
dtr = DebiasingExposure(group_col="protected",
                        query_col = 'X',
                        doc_col = 'Y',
                        feature_cols = ['score', 'protected'],
                        score_col = 'score',
                        gamma=2, 
                        number_of_iterations=100, 
                        standardize=True,
                        verbose=1)

# train the model
dtr.fit(rankings)

100%|██████████| 100/100 [00:59<00:00,  1.69it/s]


In [7]:
re_rankings = dtr.transform(rankings)

Observe the fairness metrics before and after applying the algorithm (lower is better).

In [8]:
df_deb_exp = exposure_metric(re_rankings, group_col='protected', query_col='X', score_col='score')
df_deb_exp

,Value
exposure_ratio,0.766309
exposure difference,0.002585


In [9]:
result = pd.concat([baseline, df_deb_exp], axis=1).iloc[:, [0,1]]
result.columns = ['Baseline','Mitigator']
result

,Baseline,Mitigator
exposure_ratio,33221.217929,0.766309
exposure difference,0.048336,0.002585
